# Parse HIPAA X12 transactions

## Dependencies

#### Install Pyx12

Pyx12 is a HIPAA X12 document validator and converter.
```sh
sudo pip3 install git+https://github.com/azoner/pyx12@py3-fix-scripts
```

Place X12 documents within the data directory:
```
hipaa_x12_documents/
```

### Create a list of all txt files in the data directory

In [1]:
import os
import subprocess


data_path = '../hipaa_x12_documents'


if os.path.isdir(data_path):
    print("Searching for all .txt files in: " + data_path)
    data_filepath_list = [os.path.join(data_path, filename) for filename in os.listdir(data_path) if filename.endswith(".txt")]
    if data_filepath_list:
        print('Found the following files:  ')
        for filename in data_filepath_list:
               print(filename)
    else:
        print("No data files found in path:  " + data_path) 
else:
    print("No such directory:  " + data_path)

Searching for all .txt files in: ../hipaa_x12_documents
Found the following files:  
../hipaa_x12_documents/ibc772232201701310207.txt
../hipaa_x12_documents/ibc774240201702030124.txt


### Convert to HTML and XML (optional)

In [2]:
for file in data_filepath_list:
    print("Creating HTML from: " + file)
    basename = os.path.splitext(os.path.basename(file))[0]
    cmd = 'x12html ' + file + ' > ' + os.path.join(data_path, basename) + '.html'
    print("Executing command: " + cmd)
    subprocess.call(cmd, shell=True)

    print("Creating XML from: " + file)
    basename = os.path.splitext(os.path.basename(file))[0]
    cmd = 'x12xml ' + file + ' > ' + os.path.join(data_path, basename) + '.xml'
    print("Executing command: " + cmd)
    subprocess.call(cmd, shell=True)

Creating HTML from: ../hipaa_x12_documents/ibc772232201701310207.txt
Executing command: x12html ../hipaa_x12_documents/ibc772232201701310207.txt > ../hipaa_x12_documents/ibc772232201701310207.html
Creating XML from: ../hipaa_x12_documents/ibc772232201701310207.txt
Executing command: x12xml ../hipaa_x12_documents/ibc772232201701310207.txt > ../hipaa_x12_documents/ibc772232201701310207.xml
Creating HTML from: ../hipaa_x12_documents/ibc774240201702030124.txt
Executing command: x12html ../hipaa_x12_documents/ibc774240201702030124.txt > ../hipaa_x12_documents/ibc774240201702030124.html
Creating XML from: ../hipaa_x12_documents/ibc774240201702030124.txt
Executing command: x12xml ../hipaa_x12_documents/ibc774240201702030124.txt > ../hipaa_x12_documents/ibc774240201702030124.xml


### Parse X12 documents and create separate CSV files

In [3]:
import pandas
import pyx12
import pyx12.x12context
import pyx12.params


claim_dict = {}
claim_dataframe = pandas.DataFrame()


for x12_file in data_filepath_list:
    print("Opening: " + file)
    basename = os.path.splitext(os.path.basename(x12_file))[0]
    csv_filepath = os.path.join(data_path, basename) + '.csv'
    with open(x12_file) as f:
        param = pyx12.params.params()
        errh = pyx12.error_handler.errh_null()
        src = pyx12.x12context.X12ContextReader(param, errh, f)
        for datatree in src.iter_segments('2000C'):
            for loop in datatree.select('2010CA'):
                claim_dict['last_name'] = loop.get_value('NM103')
                claim_dict['first_name'] = loop.get_value('NM104')
                claim_dict['middle_name'] = loop.get_value('NM105')
                claim_dict['unknown'] = loop.get_value('NM106')
                claim_dict['suffix_name'] = loop.get_value('NM107')
                #for loop in datatree.select('DMG'):
                claim_dict['birth_date'] = loop.get_value('DMG02')
            for loop in datatree.select('2300'):
                claim_dict['invoice'] = loop.get_value('CLM01')
                claim_dict['mrn'] = loop.get_value('REF02')
                claim_dataframe = claim_dataframe.append(claim_dict, ignore_index=True)
    print("Writing CSV: " + csv_filepath)
    claim_dataframe.to_csv(csv_filepath, index=False, mode='w', header=True)

Opening: ../hipaa_x12_documents/ibc774240201702030124.txt
Writing CSV: ../hipaa_x12_documents/ibc772232201701310207.csv
Opening: ../hipaa_x12_documents/ibc774240201702030124.txt
Writing CSV: ../hipaa_x12_documents/ibc774240201702030124.csv
